In [10]:
for (data_id in c('151507','151508','151509','151510','151669','151670','151671','151672','151673','151674','151675','151676')){

    sce_dir = paste0('./data/SpatialLIBD/',data_id,'.rds')
    save_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.h5ad.2')

    sce = readRDS(sce_dir)

    sceasy::convertFormat(sce, from="sce", to="anndata",outFile=save_dir)
    }

Warning message in .regularise_df(as.data.frame(SummarizedExperiment::colData(obj)), :
“Dropping single category variables:sample_name, tissue, height, width, subject, position, replicate, subject_position, Layer”
Warning message in .regularise_df(as.data.frame(SummarizedExperiment::rowData(obj)), :
“Dropping single category variables:type”
Warning message in .regularise_df(as.data.frame(SummarizedExperiment::colData(obj)), :
“Dropping single category variables:sample_name, tissue, height, width, subject, position, replicate, subject_position, Layer”
Warning message in .regularise_df(as.data.frame(SummarizedExperiment::rowData(obj)), :
“Dropping single category variables:type”
Warning message in .regularise_df(as.data.frame(SummarizedExperiment::colData(obj)), :
“Dropping single category variables:sample_name, tissue, height, width, subject, position, replicate, subject_position, Layer”
Warning message in .regularise_df(as.data.frame(SummarizedExperiment::rowData(obj)), :
“Dropping sin

# BayesSpace

In [1]:
library(SingleCellExperiment)
library(scuttle)
library(scran)
library(scater)
library(BiocSingular)
library(sceasy)
library(Seurat)
library(BayesSpace)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [3]:
set.seed(101)
for (data_id in c('151507','151508','151509','151510','151669','151670','151671','151672','151673','151674','151675','151676')){

    sce_dir = paste0('./data/SpatialLIBD/',data_id,'.rds')
    save_dir = paste0('./recode/IMPUTEDLIBD/BS/',data_id,'_base.out.csv')

    sce = readRDS(sce_dir)

    sce.p = spatialPreprocess(sce, platform="Visium")
    cat('Clustering','\n')
    sce.p = spatialCluster(sce.p, q=length(unique(sce$layer_guess_reordered)), d=15, platform='Visium', gamma=3)
    
    res = as.data.frame(sce.p$spatial.cluster)
    row.names(res) = colnames(sce)
    write.csv(res,save_dir)
    }

In [11]:
set.seed(101)
for (data_id in c('151673','151674','151675','151676')){
    # '151507','151508','151509','151510','151669','151670','151671','151672',
    base_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.h5ad')
    seurat_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.seurat')
    sce_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.sce')
    save_dir = paste0('./recode/IMPUTEDLIBD/BS/',data_id,'_BS.out.csv')
    cat(base_dir,'\n',seurat_dir,'\n',save_dir,'\n')

    # sceasy::convertFormat(base_dir, from="anndata", to="seurat",outFile=seurat_dir)
    # seurat.obj = readRDS(seurat_dir)
    sce = readRDS(sce_dir)
    # sceasy::convertFormat(seurat.obj, from="seurat", to="sce",outFile=sce_dir)
    df = colData(sce)
    df = cbind(df,reducedDim(sce))
    colnames(df) = c(colnames(df)[1:3],'col','row')
    sce@colData = df
    # spanve.out = seurat.obj@assays$RNA@meta.features

    cat('Clustering','\n')
    sce.i <- spatialCluster(sce, use.dimred='SPANVE.IMPUTED.PCA.PRE', q=length(unique(sce@colData$ground_truth)), d=15, platform='Visium', gamma=3)
    sce.p = spatialPreprocess(sce, platform="Visium")
    cat('Clustering','\n')
    sce.p = spatialCluster(sce.p, q=length(unique(sce@colData$ground_truth)), d=15, platform='Visium', gamma=3)
    
    # sce@assays@data$logcounts = as(t(reducedDim(sce,'S.IMPUTE.PRE.ALL')),'dgCMatrix')
    # sce.i2 = spatialPreprocess(sce, platform="Visium",log.normalize = F)
    # sce.i2 = spatialCluster(sce.i2, q=length(unique(sce@colData$ground_truth)), d=15, platform='Visium', gamma=3)
    
    res = as.data.frame(
        list(
            # spanve_i2=sce.i2$spatial.cluster,
            spanve_i=sce.i$spatial.cluster,
            base = sce.p$spatial.cluster)
        )
    write.csv(res,save_dir)
    }

./recode/IMPUTEDLIBD/processed/151673.h5ad 
 ./recode/IMPUTEDLIBD/processed/151673.seurat 
 ./recode/IMPUTEDLIBD/BS/151673_BS.out.csv 
Clustering 


Neighbors were identified for 0 out of 3639 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



Clustering 


Neighbors were identified for 0 out of 3639 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



./recode/IMPUTEDLIBD/processed/151674.h5ad 
 ./recode/IMPUTEDLIBD/processed/151674.seurat 
 ./recode/IMPUTEDLIBD/BS/151674_BS.out.csv 
Clustering 


...

Neighbors were identified for 0 out of 3592 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



./recode/IMPUTEDLIBD/processed/151676.h5ad 
 ./recode/IMPUTEDLIBD/processed/151676.seurat 
 ./recode/IMPUTEDLIBD/BS/151676_BS.out.csv 
Clustering 


Neighbors were identified for 0 out of 3460 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



Clustering 


Neighbors were identified for 0 out of 3460 spots.

Fitting model...

Calculating labels using iterations 1000 through 50000.



# SCMEB

In [1]:
library(SingleCellExperiment)
library(scuttle)
library(scran)
library(scater)
library(BiocSingular)
library(sceasy)
library(Seurat)
library(SC.MEB)
platform = "Visium"
beta_grid = seq(0,4,0.2)
K_set= 2:10
parallel=TRUE
num_core = 16
PX = TRUE
maxIter_ICM = 10
maxIter = 50

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [6]:
set.seed(101)
for (data_id in c('151507','151508','151509','151510','151669','151670','151671','151672','151673','151674','151675','151676')){

    sce_dir = paste0('./data/SpatialLIBD/',data_id,'.rds')
    save_dir = paste0('./recode/IMPUTEDLIBD/SCMEB/',data_id,'_base.out.csv')

    sce = readRDS(sce_dir)

    sce.p = spatialPreprocess(sce, platform="Visium")
    Adj_sp  <- find_neighbors2(sce, platform = "Visium")
    
    cat('Clustering','\n')
    y = reducedDim(sce.p, "PCA")[,1:15]
    fit = SC.MEB(y, Adj_sp, beta_grid = beta_grid, K_set= K_set, parallel=parallel, num_core = num_core, PX = PX, maxIter_ICM=maxIter_ICM, maxIter=maxIter)
    out = selectK(fit, K_set = K_set, criterion = "BIC")
    label = out$best_K_label
    row.names(label) = colnames(sce)
    res = as.data.frame(label)

    write.csv(res,save_dir)
    }


Attaching package: ‘purrr’


The following object is masked from ‘package:mclust’:

    map


The following object is masked from ‘package:GenomicRanges’:

    reduce


The following object is masked from ‘package:IRanges’:

    reduce



Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand


Neighbors were identified for 4226 out of 4226 spots.



Clustering 
Starting parallel computing...

Neighbors were identified for 4381 out of 4384 spots.



Clustering 
Starting parallel computing...

Neighbors were identified for 4789 out of 4789 spots.



...

Clustering 
Starting parallel computing...

Neighbors were identified for 3586 out of 3592 spots.



Clustering 
Starting parallel computing...

Neighbors were identified for 3456 out of 3460 spots.



Clustering 
Starting parallel computing...

In [2]:
for (data_id in c('151507','151508','151509','151510','151669','151670','151671','151672','151673','151674','151675','151676')){
    
    base_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.h5ad')
    seurat_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.seurat')
    sce_dir = paste0('./recode/IMPUTEDLIBD/processed/',data_id,'.sce')
    save_dir = paste0('./recode/IMPUTEDLIBD/SCMEB/',data_id,'_SCMEB.out.csv')
    cat(base_dir,'\n',seurat_dir,'\n',save_dir,'\n')

    # sceasy::convertFormat(base_dir, from="anndata", to="seurat",outFile=seurat_dir)
    seurat.obj = readRDS(seurat_dir)
    sce = readRDS(sce_dir)
    # sceasy::convertFormat(seurat.obj, from="seurat", to="sce",outFile=sce_dir)
    df = colData(sce)
    df = cbind(df,reducedDim(sce))
    colnames(df) = c(colnames(df)[1:3],'col','row')
    sce@colData = df
    spanve.out = seurat.obj@assays$RNA@meta.features
    
    Adj_sp  <- find_neighbors2(sce, platform = "Visium")
    run_scmeb = function(y){
        fit = SC.MEB(y, Adj_sp, beta_grid = beta_grid, K_set= K_set, parallel=parallel, num_core = num_core, PX = PX, maxIter_ICM=maxIter_ICM, maxIter=maxIter)
        out = selectK(fit, K_set = K_set, criterion = "BIC")
        label = out$best_K_label
        row.names(label) = colnames(sce)
        label
    }
    
    result = c()
    # ---------- Base Line
    # sce.p <- spatialPreprocess(sce, platform=platform)
    # y = reducedDim(sce.p, "PCA")[,1:15]
    # result = c(result,list(baseline=run_scmeb(y)))
    # --------- use Genes Direct
#     sce.s = sce[spanve.out$spanve_spatial_features,]
#     sce.sp = sce[spanve.out$spanve_spatial_features_pre,]
    
#     sce.s.p <- spatialPreprocess(sce.s, platform=platform)
#     y = reducedDim(sce.s.p, "PCA")[,1:15]
#     result = c(result,list(Genes=run_scmeb(y)))
    
#     sce.sp.p <- spatialPreprocess(sce.sp, platform=platform)
#     y = reducedDim(sce.sp.p, "PCA")[,1:15]
#     result = c(result,list(Genes_p=run_scmeb(y)))
    
#     # ------------ use impute
#     y = reducedDim(sce, "SPANVE.IMPUTED.PCA")[,1:15]
#     result = c(result,list(Imput=run_scmeb(y)))
    
    y = reducedDim(sce, "SPANVE.IMPUTED.PCA.PRE")[,1:15]
    result = c(result,list(Imput_p=run_scmeb(y)))
    
    write.csv(as.data.frame(result),save_dir)
}

./recode/IMPUTEDLIBD/processed/151507.h5ad 
 ./recode/IMPUTEDLIBD/processed/151507.seurat 
 ./recode/IMPUTEDLIBD/SCMEB/151507_SCMEB.out.csv 



Attaching package: ‘purrr’


The following object is masked from ‘package:mclust’:

    map


The following object is masked from ‘package:GenomicRanges’:

    reduce


The following object is masked from ‘package:IRanges’:

    reduce



Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand


Neighbors were identified for 0 out of 4226 spots.



Starting parallel computing..../recode/IMPUTEDLIBD/processed/151508.h5ad 
 ./recode/IMPUTEDLIBD/processed/151508.seurat 
 ./recode/IMPUTEDLIBD/SCMEB/151508_SCMEB.out.csv 


Neighbors were identified for 0 out of 4384 spots.



Starting parallel computing..../recode/IMPUTEDLIBD/processed/151509.h5ad 
 ./recode/IMPUTEDLIBD/processed/151509.seurat 
 ./recode/IMPUTEDLIBD/SCMEB/151509_SCMEB.out.csv 


...

Starting parallel computing..../recode/IMPUTEDLIBD/processed/151675.h5ad 
 ./recode/IMPUTEDLIBD/processed/151675.seurat 
 ./recode/IMPUTEDLIBD/SCMEB/151675_SCMEB.out.csv 


Neighbors were identified for 0 out of 3592 spots.



Starting parallel computing..../recode/IMPUTEDLIBD/processed/151676.h5ad 
 ./recode/IMPUTEDLIBD/processed/151676.seurat 
 ./recode/IMPUTEDLIBD/SCMEB/151676_SCMEB.out.csv 


Neighbors were identified for 0 out of 3460 spots.



Starting parallel computing...